In [111]:
from plotly.io import show
from sklearn.model_selection import train_test_split
from skfolio import Population
from skfolio.optimization import EqualWeighted, MaximumDiversification, Random
from skfolio.preprocessing import prices_to_returns
from openbb import obb
from itertools import chain, combinations

In [112]:
sectors = [
    "SPY",
    "QYLD",
    "XYLD",
    "ALV.DE",
    "CDR.WA",
]

In [113]:
def powerset(iterable):
    """
    powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)
    """
    xs = list(iterable)
    # note we return an iterator rather than a list
    return chain.from_iterable(combinations(xs,n) for n in range(2,len(xs)+1))

In [114]:
df = obb.equity.price.historical(
    sectors, start_date="2015-01-01", provider="yfinance"
).to_df()
pivoted = df.pivot(columns="symbol", values="close").dropna()
pivoted

C:\Users\bw200\AppData\Local\pypoetry\Cache\virtualenvs\py-quant-news-playground-0ZE7DJk8-py3.11\Lib\site-packages\yfinance\utils.py:775: FutureWarning:

The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.

C:\Users\bw200\AppData\Local\pypoetry\Cache\virtualenvs\py-quant-news-playground-0ZE7DJk8-py3.11\Lib\site-packages\yfinance\utils.py:775: FutureWarning:

The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.

C:\Users\bw200\AppData\Local\pypoetry\Cache\virtualenvs\py-quant-news-playground-0ZE7DJk8-py3.11\Lib\site-packages\yfinance\utils.py:775: FutureWarning:

The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.

C:\Users\bw200\AppData\Local\pypoetry\Cache\virtualenvs\py-quant-news-playground-0ZE7DJk8-py3.11\Lib\site-packages\yfinance\utils.py:

symbol,ALV.DE,CDR.WA,QYLD,SPY,XYLD
date,,,,,
2015-01-02,136.70,16.80,23.81,205.43,45.64
2015-01-05,133.35,16.62,23.61,201.72,44.86
2015-01-06,133.50,16.62,23.43,199.82,44.42
2015-01-07,134.75,16.84,23.55,202.31,44.93
2015-01-08,138.30,16.82,24.00,205.90,45.60
...,...,...,...,...,...
2024-02-12,246.25,113.35,17.87,500.98,40.11
2024-02-13,245.70,109.30,17.87,494.08,40.14
2024-02-14,244.60,110.30,17.89,498.57,40.15


In [115]:
X = prices_to_returns(pivoted)

X_train, X_test = train_test_split(X, test_size=0.2, shuffle=False)
X

symbol,ALV.DE,CDR.WA,QYLD,SPY,XYLD
date,,,,,
2015-01-05,-0.024506,-0.010714,-0.008400,-0.018060,-0.017090
2015-01-06,0.001125,0.000000,-0.007624,-0.009419,-0.009808
2015-01-07,0.009363,0.013237,0.005122,0.012461,0.011481
2015-01-08,0.026345,-0.001188,0.019108,0.017745,0.014912
2015-01-09,-0.017715,0.001784,-0.005833,-0.008014,-0.001754
...,...,...,...,...,...
2024-02-12,0.013374,0.019335,0.000560,-0.000439,-0.000747
2024-02-13,-0.002234,-0.035730,0.000000,-0.013773,0.000748
2024-02-14,-0.004477,0.009149,0.001119,0.009088,0.000249


In [116]:
model = MaximumDiversification()
model.fit(X_train)
ptf_model_train = model.predict(X_train)

bench = EqualWeighted()
bench.fit(X_train)
ptf_bench_train = bench.predict(X_train)

random = Random()
random.fit(X_train)
ptf_random_train = random.predict(X_train)

print(f"Maximum Diversification: {ptf_model_train.diversification:0.2f}")
print(f"Equal Weighted model: {ptf_bench_train.diversification:0.2f}")
print(f"Random Weighted model: {ptf_random_train.diversification:0.2f}")

Maximum Diversification: 1.42
Equal Weighted model: 1.40
Random Weighted model: 1.37


In [117]:
ptf_model_test = model.predict(X_test)
ptf_bench_test = bench.predict(X_test)
ptf_random_test = random.predict(X_test)
population = Population([ptf_model_test, ptf_bench_test, ptf_random_test])

In [118]:
population.plot_composition()

In [119]:
population.plot_cumulative_returns()

In [120]:
population.summary()

,MaximumDiversification,EqualWeighted,Random
Mean,0.0052%,0.011%,0.018%
Annualized Mean,1.32%,2.69%,4.66%
Variance,0.0093%,0.0095%,0.011%
Annualized Variance,2.33%,2.40%,2.89%
Semi-Variance,0.0050%,0.0051%,0.0060%
Annualized Semi-Variance,1.26%,1.28%,1.51%
Standard Deviation,0.96%,0.98%,1.07%
Annualized Standard Deviation,15.28%,15.50%,17.01%
Semi-Deviation,0.71%,0.71%,0.77%
Annualized Semi-Deviation,11.24%,11.32%,12.28%
